In [1]:
#do this later from admin: !python -m pip install --upgrade pip


In [8]:
#Basic
import pandas as pd
import numpy as np
#Math
import scipy as sp
from random import randint
from math import exp
import operator

#Scraping
import requests
import json

#API Packages: 
from apiclient.discovery import build
from apiclient.errors import HttpError
from oauth2client.tools import argparser
from googleapiclient.discovery import build
import argparse
"""

from googleapiclient.errors import HttpError"""

#Modeling
#from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
#from sklearn.linear_model import LogisticRegression
#from sklearn.naive_bayes import MultinomialNB
#from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier
#from sklearn.neighbors import KNeighborsClassifier
#from sklearn.cluster import KMeans
#from sklearn.svm import SVC
#from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve

#NLP Processing
'''from sklearn.feature_extraction import stop_words
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer'''

#Time analysis
import time
import datetime
from statsmodels.tsa.stattools import pacf
from statsmodels.graphics.tsaplots import plot_pacf

#Plotting
import matplotlib.pyplot as plt
import seaborn as sns
#from wordcloud import WordCloud

%matplotlib inline
plt.style.use('seaborn-whitegrid')

In [11]:
my_api_file = open("./mykey.txt", 'r')
my_api_str = my_api_file.read()
my_api_file.close()

In [12]:
DEVELOPER_KEY = my_api_str
YOUTUBE_API_SERVICE_NAME = "youtube"
YOUTUBE_API_VERSION = "v3"

In [ ]:
# all views over 400M https://www.youtube.com/playlist?list=PLirAqAtl_h2r5g8xGajEwdXd3x1sZh8hC

https://www.youtube.com/results?filters=video&lclk=video&search_query=a%7Cb%7Cc%7Cd%7Ce%7Cf%7Cg%7Ch%7Ci%7Cj%7Ck%7Cl%7Cm%7Cn%7Co%7Cp%7Cq%7Cr%7Cs%7Ct%7Cu%7Cv%7Cw%7Cx%7Cy%7Cz%7C1%7C1%7C2%7C3%7C4%7C5%7C6%7C7%7C8%7C9%7C0&search_sort=video_view_count


YouTube Search_list filtering parameters (sort_method):
* date – Resources are sorted in reverse chronological order based on the date they were created.
* rating – Resources are sorted from highest to lowest rating.
* relevance – Resources are sorted based on their relevance to the search query. This is the default value for this parameter.
* title – Resources are sorted alphabetically by title.
* videoCount – Channels are sorted in descending order of their number of uploaded videos.
* viewCount – Resources are sorted from highest to lowest number of views. For live broadcasts, videos are sorted by number of concurrent viewers while the broadcasts are ongoing.

In [27]:
'''Hold off on this one until I figure out how to tie in google trends. 
Probably we want only comments from peak viewership plus marks at half peak on either side. 
Right now, most relevant /recent comments are all stupid meme stuff, e.g. who is watching in 2018'''

#This function will grab 50 comments at a time from the video. 
def youtube_comment_list(video_id, max_results=50, sort_method='viewCount', token=None):
  # Call the search.list method to retrieve results matching the specified query term.
    youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION,
    developerKey=DEVELOPER_KEY)

  # Call the search.list method to retrieve results matching the specified query term.
    search_response = youtube.commentThreads().list(
        videoId = video_id,
        part='snippet, replies',
        pageToken=token,
        order = 'relevance',
        maxResults=max_results,
      ).execute()
    
    return search_response

In [28]:
psy = youtube_comment_list('9bZkp7q19f0')

psy['items'][1]['snippet']['topLevelComment']['snippet']['textDisplay']

'Sep 24th?'

## Get a bunch of data, see what happens

In [20]:
df_videos, next_tok = youtube_search_video_looper(q='music video',max_results=50, 
                                sort_method='relevance', 
                                token=None, 
                                dl_path='./data/music_videos_relevance/', dl_title='iteration2')

    found page  1
    found page  2
    found page  3
    found page  4
    found page  5
    found page  6
    found page  7
    found page  8
    found page  9
    found page  10
    found page  11
    found page  12
    found page  13
    found page  14
    found page  15
    found page  16
    found page  17
    found page  18


good idea to set up a basic cleaning function for these - probably will do a lot

In [22]:
#df_lyrics_viewCount lyric video search by viewcount
#df_mv_relevance music video search by relevance


In [30]:
df_mv_relevance['viewCount'] = df_mv_relevance['viewCount'].map(lambda x: 0 if x=='xxNoneFoundxx' else x)

In [37]:
df_mv_relevance['viewCount'] = df_mv_relevance['viewCount'].astype('int64')

In [74]:
#duplicates
pd.concat([df_mv_viewCount.loc[:, ['title']],  
           df_mv_relevance.loc[:, ['title']]]).sort_values(by='title').duplicated().sum()/2

56.0

In [76]:
#Less duplicates, retrieved 300 new videos
df_mv_relevance.shape[0] - \
pd.concat([df_mv_viewCount.loc[:, ['title']],  
           df_mv_relevance.loc[:, ['title']]]).sort_values(by='title').duplicated().sum()/2

306.0

In [18]:
def youtube_search_cat(q, max_results=50, sort_method='viewCount', token=None):
    # Call the search.list method to retrieve results matching the specified query term.
    youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION,
    developerKey=DEVELOPER_KEY)
    
    search_response = youtube.search().list(
        q=q,
        part='id,snippet',
        pageToken=token,
        maxResults=max_results,
        order=sort_method,
        videoCategoryId='10',
        type='video'
    ).execute()
    
    return search_response

In [20]:
#takes output json list from search list and returns metadata on individual videos
def youtube_search_video_cat(q='spinners', max_results=50, sort_method='viewCount', token=None):
    
    order = "viewCount"
    q=q
    max_results = max_results
    sort_method = sort_method
    token = token
    location = None
    location_radius = None
    
    youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION,         
    developerKey=DEVELOPER_KEY)
    
    #Return list of matching records up to max_search
    search_result = youtube_search_cat(q, max_results, sort_method, token)
    next_tok = search_result['nextPageToken']
    
    videos_list = []
    for search_result in search_result.get("items", []):
        
        if search_result["id"]["kind"] == 'youtube#video':
            temp_dict_ = {}
            #Available from initial search
            temp_dict_['title'] = search_result['snippet']['title']  
            temp_dict_['vidId'] = search_result['id']['videoId']  
            
            #Secondary call to find statistics results for individual video
            response = youtube.videos().list(
                part='statistics, snippet, contentDetails', 
                id=search_result['id']['videoId']
                    ).execute()
            #Relevant dictionaries
            response_statistics = response['items'][0]['statistics']
            response_snippet = response['items'][0]['snippet']
            response_content= response['items'][0]['contentDetails']
            
            
            snippet_list = ['publishedAt','channelId', 'description', 
                            'channelTitle', 'tags', 'categoryId', 
                            'liveBroadcastContent', 'defaultLanguage', ]
            for val in snippet_list:
                try:
                    temp_dict_[val] = response_snippet[val]
                except:
                    #Not stored if not present
                    temp_dict_[val] = 'xxNoneFoundxx'    
            
            stats_list = ['favoriteCount', 'viewCount', 'likeCount', 
                          'dislikeCount', 'commentCount']
            for val in stats_list:
                try:
                    temp_dict_[val] = response_statistics[val]
                except:
                    #Not stored if not present
                    temp_dict_[val] = 'xxNoneFoundxx'
            
            for val in response_content.keys():
                try:
                    temp_dict_[val] = response_content[val]
                except:
                    #Not stored if not present
                    temp_dict_[val] = 'xxNoneFoundxx'
                    
            #add back to main list
            videos_list.append(temp_dict_)
            
    return videos_list, next_tok

In [33]:
#Automation
def youtube_search_video_looper_cat(q,max_results=50, 
                                sort_method='viewCount', 
                                token=None, 
                                dl_path='./data/test/', dl_title='iteration'):
    #find relevant stats, create dataframe of results, create running csvs
    q=q,
    max_results=max_results
    sort_method = sort_method
    token = token
    
    #create unique text file name
    now_month = datetime.datetime.now().month
    now_day = datetime.datetime.now().day
    now_hour = datetime.datetime.now().hour
    now_minute = datetime.datetime.now().minute
    output_title = '{}{}_{}.{}_{}{}.csv'.format(dl_path, dl_title,  
                                                  now_month, now_day, now_hour, now_minute)
    
    
    #Cool Start to get next token
    videos_list, next_tok = youtube_search_video_cat(q=q,
                                                 max_results=max_results, 
                                                 sort_method=sort_method, 
                                                 token=token)
    df_videos = pd.DataFrame(videos_list)
    df_videos['request_token'] = next_tok #for assessment of run.

    #Set up exit criteria - 2 counts in a row means its done
    previous_results = 0
    current_results = df_videos.shape[0]
    
    count=1
    while previous_results != current_results:
        try:

            videos_list, next_tok = youtube_search_video_cat(q,
                                                         max_results=max_results, 
                                                         sort_method=sort_method, 
                                                         token=next_tok)
            print('    found page ', count)
            df_small_vids = pd.DataFrame(videos_list)
            df_small_vids['request_token'] = next_tok
            df_videos = pd.concat([df_videos, df_small_vids], sort=False)


            #save as we go
            df_videos.to_csv(output_title, index=False)

            _temp = current_results
            current_results = df_videos.shape[0]
            previous_results = _temp
            count += 1
            time.sleep(2)
        except:
            break
    total_pulls = count
    return df_videos, next_tok, total_pulls
        

In [26]:
letters = 'abcdefghijklmnopqrxtuvwxyz'

In [31]:
df_big_videos = pd.DataFrame()

In [35]:
letters[0] + 'iteration'

'aiteration'

In [36]:
count=0
for letter in letters:
    df_videos, next_tok, total_pulls = youtube_search_video_looper_cat(q=letter, sort_method='relevance', dl_path='./data/cat10_relevance/', dl_title='search'+letter)
    df_videos['letter_search'] = letter
    print('   foundall: ', letter)
    df_big_videos = pd.concat([df_big_videos, df_videos], sort=False)
    df_big_videos.to_csv('./data/cat10_relevance/bigletters.csv')
    

    found page  1
    found page  2
    found page  3
    found page  4
    found page  5
    found page  6
    found page  7
   foundall:  a
    found page  1
    found page  2
    found page  3
    found page  4
    found page  5
    found page  6
    found page  7
    found page  8
    found page  9
   foundall:  b
    found page  1
    found page  2
    found page  3
    found page  4
    found page  5
    found page  6
    found page  7
    found page  8
    found page  9
   foundall:  c
    found page  1
    found page  2
    found page  3
    found page  4
    found page  5
    found page  6
    found page  7
    found page  8
   foundall:  d
    found page  1
    found page  2
    found page  3
    found page  4
    found page  5
    found page  6
    found page  7
    found page  8
   foundall:  e
    found page  1
    found page  2
    found page  3
    found page  4
    found page  5
    found page  6
    found page  7
    found page  8
    found page  9
    found page  10
 

In [38]:
df_big_videos.shape

(10259, 25)

# Clean up

In [103]:
df_new = df_big_videos

In [104]:
df_new = df_new.reset_index()

In [105]:
df_new = df_new.drop('index', axis=1)

In [106]:
df_new.columns

Index(['caption', 'categoryId', 'channelId', 'channelTitle', 'commentCount',
       'defaultLanguage', 'definition', 'description', 'dimension',
       'dislikeCount', 'duration', 'favoriteCount', 'licensedContent',
       'likeCount', 'liveBroadcastContent', 'projection', 'publishedAt',
       'regionRestriction', 'tags', 'title', 'vidId', 'viewCount',
       'request_token', 'letter_search', 'contentRating'],
      dtype='object')

In [107]:
df_new = df_new.drop_duplicates(subset='vidId')

In [108]:
df_new[df_new.commentCount == 'xxNoneFoundxx']

,caption,categoryId,channelId,channelTitle,commentCount,defaultLanguage,definition,description,dimension,dislikeCount,...,projection,publishedAt,regionRestriction,tags,title,vidId,viewCount,request_token,letter_search,contentRating
60,false,10,UCYMN4uQ_gnzpWKcla035dWQ,K&K Philharmoniker,xxNoneFoundxx,de,sd,"Giuseppe Verdi - Nabucco - Va, Pensiero (Gefan...",2d,2261,...,rectangular,2009-10-27T14:05:25.000Z,NaN,"[Chorus, of, the, Hebrew, Slaves, Nabucco, Die...","Va, pensiero (Gefangenenchor) · Giuseppe Verdi...",XttF0vg0MGo,9905401,CGQQAA,a,NaN
146,false,10,UCzOW2pM5n2KTe8bvll64lew,Davide Consales1,xxNoneFoundxx,xxNoneFoundxx,hd,,2d,5450,...,rectangular,2012-03-22T18:59:31.000Z,NaN,"[Tina Turner, Proud Mary, creedence]",Tina turner- Proud Mary,02QUmKVsyFY,20568616,CJYBEAA,a,NaN
191,false,10,UCkfCix1fesR6v0tE-fIweNg,miljkmi,xxNoneFoundxx,xxNoneFoundxx,hd,"Ralph Vaughan Williams (1872-1958), England\n\...",2d,205,...,rectangular,2012-03-19T18:31:50.000Z,NaN,"[Ralph, Vaughn, Williams, England, Fantasia, o...",Ralph Vaughan Williams - Fantasia on a Theme b...,0U6sWqfrnTs,1668228,CMgBEAA,a,NaN
238,false,10,UCz2vvYornA6zuhIQRS6jTFg,facevideobook,xxNoneFoundxx,xxNoneFoundxx,hd,"The Piano Sonata No.14 ""Quasi Una Fantasia"" Op...",2d,xxNoneFoundxx,...,rectangular,2013-04-18T16:52:16.000Z,NaN,"[valentina lisitsa, ludwig van beethoven, moon...",Valentina Lisitsa - Moonlight Sonata Op.27 No....,OsOUcikyGRk,6057735,CPoBEAA,a,NaN
273,false,10,UCGF_z7NNup6feP3ry_mRxnA,YaleCampus,xxNoneFoundxx,xxNoneFoundxx,hd,"In February 2010, Anna Gawboy, a doctoral cand...",2d,xxNoneFoundxx,...,rectangular,2010-09-14T18:46:33.000Z,NaN,"[prometheus, poem of fire, alexander scriabin,...",Scriabin's Prometheus: Poem of Fire,V3B7uQ5K0IU,190235,CKwCEAA,a,NaN
298,false,10,UCT9iwdHG-u--1cUCktyeuMg,shelim23,xxNoneFoundxx,xxNoneFoundxx,hd,"1) Piano Sonata No.16 in C major, K. 545, I. ...",2d,3,...,rectangular,2016-01-16T13:27:18.000Z,NaN,"[Wolfgang Amadeus Mozart, Masashi hamauzu, Son...","Piano Sonata No 16 in C major, K 545, I All...",GSKqyF5UtH4,5814,CKwCEAA,a,NaN
676,false,10,UCh9Rp9B9H0DTtkhdqdtbZ8Q,Das Flug,xxNoneFoundxx,xxNoneFoundxx,hd,leleks obeyen likebefehl: http://www.facebook....,2d,277,...,rectangular,2012-02-12T19:12:14.000Z,NaN,"[Das Flug, Boeing B-17 Flying Fortress, Origin...",Das Flug - Alles musz in Flammen stehen (B-17 ...,W_oJf54ZoRE,392161,CJADEAA,b,NaN
714,false,10,UCZE7PatIP-m78XQfI3b_TYw,yossy+,xxNoneFoundxx,xxNoneFoundxx,sd,"Karl Leister, Tomoko Kato, Akihiro Miura, Yosh...",2d,27,...,rectangular,2014-10-03T21:58:44.000Z,NaN,"[Johannes Brahms (Composer), Clarinet Quintet ...","Brahms Clarinet Quintet in B minor, Op.115",qBS7BC4aKGE,109309,CJADEAA,b,NaN
1035,false,10,UCzEuGgnpqqnBGlaOtVRxkRA,mahlerite,xxNoneFoundxx,xxNoneFoundxx,sd,Wolfgang Amadeus Mozart \r\nChristoph Eschenbach,2d,539,...,rectangular,2008-12-01T05:26:26.000Z,NaN,"[Allegro, Piano, Sonata, 16, Facile, Semplice]","Mozart Piano Sonata in C, K. 545 (1/2); 1st mo...",JcUh-ggBfzI,4781855,CKwCEAA,c,NaN
1184,false,10,UCaZrRRS-39I6UPmiwOP2lLg,menifred,xxNoneFoundxx,xxNoneFoundxx,hd,BBC Proms 2013 - Conductor: Paavo Järvi,2d,xxNoneFoundxx,...,rectangular,2014-03-05T04:28:00.000Z,NaN,"[Camille Saint-Saëns (Musical Artist), Organ (...","Saint-Saëns - Symphony No 3 in C minor, Op 78,...",M68gT9XQMEw,44740,CMIDEAA,c,NaN


In [109]:
df_new['commentCount'] = df_new['commentCount'].map(lambda x: 0 if x == 'xxNoneFoundxx' else x )

In [110]:
df_new['commentCount'] =df_new['commentCount'].astype('str').astype('int')

In [111]:
df_new['commentCount'].sum()

168750804

In [112]:
def clean_df_new_col(col):
    df_new[col] = df_new[col].map(lambda x: 0 if x == 'xxNoneFoundxx' else x )
    df_new[col] =df_new[col].astype('str').astype('int64')

In [113]:
clean_df_new_col('favoriteCount')

In [114]:
clean_df_new_col('likeCount')

In [115]:
clean_df_new_col('viewCount')

In [116]:
clean_df_new_col('dislikeCount')

In [117]:
df_new.dtypes

caption                 object
categoryId              object
channelId               object
channelTitle            object
commentCount             int32
defaultLanguage         object
definition              object
description             object
dimension               object
dislikeCount             int64
duration                object
favoriteCount            int64
licensedContent         object
likeCount                int64
liveBroadcastContent    object
projection              object
publishedAt             object
regionRestriction       object
tags                    object
title                   object
vidId                   object
viewCount                int64
request_token           object
letter_search           object
contentRating           object
dtype: object

In [119]:
df_new['tags'] = df_new.tags.apply(lambda x: ', '.join(x) if x != 'xxNoneFoundxx' else '')

In [153]:
df_new['channelTitle'].value_counts()

RKellyVEVO                 57
WORLDSTARHIPHOP            49
SMTOWN                     46
Canal KondZilla            35
PinkVEVO                   33
jbalvinVEVO                32
T-Series                   32
BeckyGVEVO                 32
Sara'h Officiel            30
GEazyMusicVEVO             28
1theK (원더케이)               27
avex                       27
Ceylin - H Official        25
Peder B. Helland           24
Ultra Music                22
Various Artists - Topic    21
Fueled By Ramen            21
jypentertainment           21
GunsNRosesVEVO             20
KarolGVEVO                 18
BIGBANG                    18
Luccas Neto                18
OfficialCKanVEVO           18
JFlaMusic                  18
ScHoolBoyQVEVO             17
Q Money                    17
starshipTV                 17
GR6 EXPLODE                16
JessieJVEVO                16
Am4d3usM0z4rt              16
                           ..
山岸シュンスケ《なりきりユーミン》           1
Mona Zidany                 1
miglioridi

In [154]:
df_new.to_csv('./data/alphabet_pull_init_01.10.18.csv', index=False)

# To Do
* Convert 
 * Duration
 * PublishedAt
* Get dummies: 
 * definition, 
 * defaultLanguage, 
 * licensedContent
 * Caption
* Drop:
 * Projection, 
 * Dimension
 * 3d movies,
 * livebroadcast
 * CategoryID
 * channelID (already have title, which has text!!)
 * LiveBroadCastContent

# Leaving in
* Region restriction should be dropped until modeling - consider bringing this back later
* ContentRating - insignificant but interesting
* Could look at channelTitle
* request_token
* letter_search

# Anticipated future data gathering
* Top 10 comment strings
* View Count by date